In [1]:
!pip install chess
import re
import chess
import random
import pandas as pd
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

  Obtaining dependency information for chess from https://files.pythonhosted.org/packages/d6/d8/15cfcb738d2518daf04d34b23419bd359cbd8e09da50778ebac521774fc8/chess-1.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.2 MB/s eta 0:00:00


In [ ]:
board = chess.Board()
board

In [ ]:
def get_mapping(board, piece):
    s = str(board)
    s = re.sub(f'[^{piece}{piece.upper()} \n]', '.', str(board))
    s = re.sub(f'{piece}', '-1', s)
    s = re.sub(f'{piece.upper()}', '1', s)
    s = re.sub(f'\.', '0', s)
    
    matrix = []
    for row in s.split('\n'):
        row = row.split(' ')
        row = [int(x) for x in row]
        matrix.append(row)
    return matrix

torch.tensor(get_mapping(board, 'p'))

In [ ]:
def board_repr(board):
    pieces = ['p', 'r', 'n', 'b', 'q', 'k']
    layers = []
    for piece in pieces:
        layers.append(get_mapping(board, piece))
    return torch.tensor(layers)

board_repr(board).shape

In [ ]:
ltr_to_num = {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7}
num_to_ltr = {0: 'a', 1: 'b', 2: 'c', 3: 'd', 4: 'e', 5: 'f', 6: 'g', 7: 'h'}

In [ ]:
def move_repr(move):
    start = [[0] * 8 for _ in range(8)]
    start_row = 8 - int(move[1])
    start_col = ltr_to_num[move[0]]
    start[start_row][start_col] = 1
    
    end = [[0] * 8 for _ in range(8)]
    end_row = 8 - int(move[3])
    end_col = ltr_to_num[move[0]]
    end[end_row][end_col] = 1
    
    return torch.tensor([start, end])

move_repr('d2d4')

In [ ]:
class Block(nn.Module):
    
    def __init__(self, hidden_size):
        super(Block, self).__init__()
        self.conv1 = nn.Conv2d(hidden_size, hidden_size, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(hidden_size, hidden_size, kernel_size=3, stride=1, padding=1)
        self.batch_norm = nn.BatchNorm2d(hidden_size)
    
    def forward(self, x):
        x_copy = torch.clone(x)
        x = F.selu(self.batch_norm(self.conv1(x)))
        x = F.selu(self.batch_norm(self.conv2(x)) + x_copy)
        return x
    
class ChessNet(nn.Module):
    
    def __init__(self, input_size=6, output_size=2, hidden_size=100, num_blocks=10):
        super(ChessNet, self).__init__()
        self.input = nn.Conv2d(input_size, hidden_size, kernel_size=3, stride=1, padding=1)
        self.blocks = nn.ModuleList([Block(hidden_size) for _ in range(num_blocks)])
        self.output = nn.Conv2d(hidden_size, output_size, kernel_size=3, stride=1, padding=1)
    
    def forward(self, x):
        x = F.selu(self.input(x))
        for block in self.blocks:
            x = block(x)
        x = self.output(x)
        return x

model = ChessNet().cuda()
model

In [19]:
class ChessDataset(Dataset):
    
    def __init__(self, dataframe):
        super(ChessDataset, self).__init__()
        self.boards = dataframe['states']
        self.moves = dataframe['moves']
        self.player = dataframe['player']
    
    def __len__(self):
        return len(self.moves)
    
    def __getitem__(self, index):
        board = chess.Board(self.boards[index])
        next_move = self.moves[index]
        player = self.player[index]
        x = board_repr(board)
        y = move_repr(next_move)
        if player == 1:
            x *= -1
        x = x.type(torch.float32)
        y = y.type(torch.float32)
        return x, y

In [ ]:
dataset = pd.read_csv()

In [17]:
length, ratio = len(dataset), 0.9
train_dataset = dataset[:int(ratio*length)]
test_dataset = dataset[int(ratio*length):]
len(train_dataset), len(test_dataset)

(459, 51)

In [20]:
train_data = ChessDataset(train_dataset)
test_data = ChessDataset(test_dataset)
train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True, drop_last=True)

In [ ]:
loss_fn1 = nn.CrossEntropyLoss()
loss_fn2 = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=model.parameters(), lr=0.01)

In [ ]:
epochs = 1
for epoch in tqdm(range(epochs)):
    print(f"epoch -> {epoch}")
    train_loss = 0
    print(f"training....")
    for x, y in tqdm(train_dataloader):
        x, y = x.cuda(), y.cuda()
        model.train()
        y_pred = model(x)
        
        loss1 = loss_fn1(y_pred[:, 0, :], y[:, 0, :])
        loss2 = loss_fn2(y_pred[:, 1, :], y[:, 1, :])
        loss = loss1 + loss2
        train_loss += loss
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss /= len(train_dataloader)
    
    test_loss = 0
    model.eval()
    print(f"testing....")
    with torch.inference_mode():
        for x, y in tqdm(test_dataloader):
            x, y = x.cuda(), y.cuda()
            y_pred = model(x)
            loss1 = loss_fn1(y_pred[:, 0, :], y[:, 0, :])
            loss2 = loss_fn2(y_pred[:, 1, :], y[:, 1, :])
            loss = loss1 + loss2
            test_loss += loss
    
        test_loss /= len(test_dataloader)
        
    print(train_loss)
    print(test_loss)